### Process outline

1. Follow the scVI tutorial: https://docs.scvi-tools.org/en/stable/tutorials/notebooks/scrna/harmonization.html

In [1]:
import os

import scanpy as sc
import scvi
import seaborn as sns
import torch
from rich import print
from scib_metrics.benchmark import Benchmarker

import warnings
warnings.filterwarnings("ignore")

In [2]:
scvi.settings.seed = 0 # for reproducibility
print("Last run with scvi-tools version:", scvi.__version__)

[rank: 0] Seed set to 0


Last run with scvi-tools version: 1.1.2

In [3]:
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

In [4]:
# Load all AnnData objects into a list

from pathlib import Path
from itertools import chain

GSE132509_directory = Path('/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE132509')
GSE236351_directory = Path('/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE236351')
GSE148218_directory = Path('/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE148218')

combined_dirs = chain(GSE132509_directory.iterdir(), GSE236351_directory.iterdir(), GSE148218_directory.iterdir())
adatas = []
for adata_path in combined_dirs:
    if "_uni.h5ad" in adata_path.name:
        adata = sc.read_h5ad(adata_path)
        adatas.append(adata)

print(len(adatas))
print(adatas[0])

26

AnnData object with n_obs × n_vars = 2229 × 33694
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
    var: 'gene_ids'

In [5]:
# Find out common genes among all AnnData objects
common_genes = set(adatas[0].var_names)
for adata in adatas[1:]:
    common_genes.intersection_update(adata.var_names)

print(len(common_genes))

14071

In [6]:
# Filter all AnnData objects with common genes
adatas_common_genes = []
for adata in adatas:
    adata_common_genes = adata[:, list(common_genes)]
    print(adata_common_genes.shape)
    adatas_common_genes.append(adata_common_genes)

(2229, 14071)

(1612, 14071)

(2748, 14071)

(3105, 14071)

(5069, 14071)

...

(7917, 14071)

(7160, 14071)

(4224, 14071)

(6106, 14071)

(7228, 14071)

### <span style="color:yellow">**Preprocessing:**</span> normalization & log transformation

Follow the scanpy preprocessing tutorial: https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

Use the preprocessing package from dandelion to filter out cell and gene outliers

In [8]:
from dandelion.preprocessing.external._preprocessing import recipe_scanpy_qc

adatas_filtered = [] 

for adata in adatas_common_genes:
    adata.raw = adata
    print(adata.shape)

    # Do QC and filtering
    recipe_scanpy_qc(adata)
    adata = adata[adata.obs.filter_rna == 'False', :]
    print(adata.shape)

    # Do normalization
    sc.pp.normalize_total(adata)

    # Do the log transformation
    sc.pp.log1p(adata)

    adatas_filtered.append(adata)

(2229, 14071)

(1536, 14071)

(1612, 14071)

(1328, 14071)

(2748, 14071)

...

(2552, 14071)

(6106, 14071)

(3996, 14071)

(7228, 14071)

(5105, 14071)

In [9]:
# Create a merged AnnData for all filtered Anndata objects
adatas_filtered_all = sc.AnnData.concatenate(*adatas_filtered)

In [10]:
adatas_filtered_all

AnnData object with n_obs × n_vars = 64278 × 14071
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'scrublet_score', 'is_doublet', 'filter_rna', 'batch'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-10', 'gene_ids-11', 'feature_types-11', 'gene_ids-12', 'feature_types-12', 'gene_ids-13', 'feature_types-13', 'gene_ids-14', 'feature_types-14', 'gene_ids-15', 'feature_types-15', 'gene_ids-16', 'feature_types-16', 'gene_ids-17', 'feature_types-17', 'gene_ids-18', 'feature_types-18', 'gene_ids-19', 'feature_types-19', 'gene_ids-2', 'gene_ids-20', 'feature_types-20', 'gene_ids-21', 'feature_types-21', 'gene_ids-22', 'feature_types-22', 'gene_ids-23', 'feature_types-23', 'gene_ids-24', 'feature_types-24', 'gene_ids-25', 'feature_types-25', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'gene_ids-7', 'gene_ids-8', 'gene_ids-9'

In [12]:
# Select highly variable genes
sc.pp.highly_variable_genes(
    adatas_filtered_all,
    flavor="seurat_v3",
    n_top_genes=2000,
    batch_key="sample_barcode",
    subset=True,
)

In [13]:
adatas_filtered_all

AnnData object with n_obs × n_vars = 64278 × 2000
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'scrublet_score', 'is_doublet', 'filter_rna', 'batch'
    var: 'gene_ids-0', 'gene_ids-1', 'gene_ids-10', 'gene_ids-11', 'feature_types-11', 'gene_ids-12', 'feature_types-12', 'gene_ids-13', 'feature_types-13', 'gene_ids-14', 'feature_types-14', 'gene_ids-15', 'feature_types-15', 'gene_ids-16', 'feature_types-16', 'gene_ids-17', 'feature_types-17', 'gene_ids-18', 'feature_types-18', 'gene_ids-19', 'feature_types-19', 'gene_ids-2', 'gene_ids-20', 'feature_types-20', 'gene_ids-21', 'feature_types-21', 'gene_ids-22', 'feature_types-22', 'gene_ids-23', 'feature_types-23', 'gene_ids-24', 'feature_types-24', 'gene_ids-25', 'feature_types-25', 'gene_ids-3', 'gene_ids-4', 'gene_ids-5', 'gene_ids-6', 'gene_ids-7', 'gene_ids-8', 'gene_ids-9', 'highly_variable', 'highly_variable_rank', 'm

### <span style="color:yellow">**Attempt 2:**</span> concatenate all AnnData objects and find highly variable genes together

In [ ]:
# Concatenate filtered AnnData objects into one to select highly variable genes
adatas_filtered_all = sc.AnnData.concatenate(*adatas_filtered)

target_genes = 1000
sc.pp.highly_variable_genes(adatas_filtered_all, n_top_genes=target_genes, batch_key='dataset')
adatas_filtered_all

In [ ]:
adatas_filtered_all.obs

In [ ]:
# As we don't have enough target genes, we need to consider the 'next best' HVGs
n_batches = len(adatas_filtered_all.obs['sample_barcode'].cat.categories)

# These are the genes that are variable across all batches
nbatch1_dispersions = adatas_filtered_all.var['dispersions_norm'][adatas_filtered_all.var.highly_variable_nbatches > n_batches - 1]
nbatch1_dispersions.sort_values(ascending=False, inplace=True)
print(len(nbatch1_dispersions))

In [ ]:
# Fill up the genes now, using this method from the Theis lab
enough = False
hvg = nbatch1_dispersions.index[:]
not_n_batches = 1

# We'll go down one by one, until we're selecting HVGs from just a single batch.
while not enough:

    target_genes_diff = target_genes - len(hvg) # Get the number of genes we still need to fill up

    tmp_dispersions = adatas_filtered_all.var['dispersions_norm'][adatas_filtered_all.var.highly_variable_nbatches == (n_batches - not_n_batches)]

    # If we haven't hit the target gene numbers, add this to the list and we repeat this iteration
    if len(tmp_dispersions) < target_genes_diff:

        hvg = hvg.append(tmp_dispersions.index)
        not_n_batches += 1

    else:

        tmp_dispersions.sort_values(ascending=False, inplace=True)
        hvg = hvg.append(tmp_dispersions.index[:target_genes_diff])
        enough = True

In [ ]:
# Subset the meta AnnData object on the highly variable genes
adatas_filtered_hvg_all = adatas_filtered_all[:, hvg]

In [ ]:
adatas_filtered_hvg_all

In [ ]:
# Visualize the data before integration
sc.tl.pca(adatas_filtered_hvg_all) # Calculate the PCA embeddings
sc.pp.neighbors(adatas_filtered_hvg_all) # Determine the kNN graph
sc.tl.umap(adatas_filtered_hvg_all) # Calculate the UMAP

In [ ]:
sc.pl.umap(adatas_filtered_hvg_all, color=['dataset'])
sc.pl.umap(adatas_filtered_hvg_all, color=['sample_barcode'])

In [ ]:
# Split the meta AnnData
adatas_filtered_hvg = []

for batch in adatas_filtered_hvg_all.obs['batch'].unique():
    adatas_filtered_hvg.append(adatas_filtered_hvg_all[adatas_filtered_hvg_all.obs['batch']==batch].copy())

print(len(adatas_filtered_hvg))
print(adatas_filtered_hvg[0])

In [ ]:
# Now we run Scanorama on the split data.
import scanorama

corrected = scanorama.correct_scanpy(adatas_filtered_hvg, return_dimred=True)

# Concatenate the integrated AnnData objects
adata_integrated = sc.AnnData.concatenate(*corrected)
print(adata_integrated)

In [ ]:
# Do the UMAP to visualize the integration results
sc.pp.neighbors(adata_integrated, use_rep='X_scanorama')
sc.tl.umap(adata_integrated)

In [ ]:
sc.pl.umap(adata_integrated, color=['dataset'])
sc.pl.umap(adata_integrated, color=['sample_barcode'])

### <span style="color:yellow">**Attempt 3:**</span> concatenate all AnnData objects and find highly variable genes together and scale individually

In [ ]:
# The preprocessing is the same as Attempt 2,
# but before integration, we scale the gene expression of the meta AnnData

sc.pp.scale(adatas_filtered_hvg_all, max_value=10)

# Visualize the data before integration
sc.tl.pca(adatas_filtered_hvg_all) # Calculate the PCA embeddings
sc.pp.neighbors(adatas_filtered_hvg_all) # Determine the kNN graph
sc.tl.umap(adatas_filtered_hvg_all) # Calculate the UMAP

In [ ]:
sc.pl.umap(adatas_filtered_hvg_all, color=['dataset'])
sc.pl.umap(adatas_filtered_hvg_all, color=['sample_barcode'])

In [ ]:
# Split the meta AnnData
adatas_filtered_hvg_scaled = []

for batch in adatas_filtered_hvg_all.obs['batch'].unique():
    adatas_filtered_hvg_scaled.append(adatas_filtered_hvg_all[adatas_filtered_hvg_all.obs['batch']==batch].copy())

print(len(adatas_filtered_hvg_scaled))
print(adatas_filtered_hvg_scaled[0])

In [ ]:
# Now we run Scanorama on the split data.
import scanorama

corrected = scanorama.correct_scanpy(adatas_filtered_hvg_scaled, return_dimred=True)

# Concatenate the integrated AnnData objects
adata_integrated_scaled = sc.AnnData.concatenate(*corrected)
print(adata_integrated_scaled)

In [ ]:
# Do the UMAP to visualize the integration results
sc.pp.neighbors(adata_integrated_scaled, use_rep='X_scanorama')
sc.tl.umap(adata_integrated_scaled)

In [ ]:
sc.pl.umap(adata_integrated_scaled, color=['dataset'])
sc.pl.umap(adata_integrated_scaled, color=['sample_barcode'])